In [1]:
from parsel import Selector
from lxml import etree
from lxml.etree import tostring
from itertools import chain

import json

In [3]:
def get_text(node):
    
    if isinstance(node, str):
        if len(node.strip()) > 0:
            return [node]
        else:
            return []
    if node == None:
        return []
    
    txt_lst = []
    for c in node.getchildren():
        txt_lst_node = get_text(c)
        if txt_lst_node != []:
            txt_lst += txt_lst_node
        
        txt_lst_tail = get_text(c.tail)
        if txt_lst_tail != []:
            txt_lst += txt_lst_tail
        
    return txt_lst

    

In [4]:
import os

In [5]:
output_dir = "data_profiles_json/"
for dirpath, dirname, filelist in os.walk("data_profiles_html/"):
    for f in filelist:
        if f.endswith(".html"):
            
            full_name = os.path.join(dirpath, f)
            
            data_person = {}
            
            with open(full_name, "r") as fd:
                print("-->", full_name)
                
                sel = Selector(text=fd.read())
                
                name = sel.xpath("//*[starts-with(@id,\"ember\")]/div[2]/div[2]/div[1]/div[1]/h1/text()").extract_first()
                current_position = sel.xpath("//*[starts-with(@id,\"ember\")]/div[2]/div[2]/div[1]/div[2]/text()").extract_first()
                location = sel.xpath("//*[starts-with(@id,\"ember\")]/div[2]/div[2]/div[2]/span[1]/text()").extract_first()
                
                if name != None:
                    name = name.strip()
                if current_position != None:
                    current_position = current_position.strip()
                if location != None:
                    location = location.strip()
                print(name)
                    
                if name != None:
                    
                    data_person["name"] = name
                    data_person["current_position"] = current_position
                    data_person["location"] = location
                
                    #//*[@id="ember113"]/div[3]/div/div/div/span[1]
                
                    about = sel.xpath("//*[starts-with(@id,\"ember\")]/div[3]/div/div/div/span[1]/text()").extract_first()
                    if about != None:
                        about = about.strip()
                    
                    
                
                    #//*[@id="ember115"]/div[3]/ul
                    # See how to get experience as a list: https://parsel.readthedocs.io/en/latest/usage.html
                    
                    #'attrib', 'css', 'drop', 'extract', 'get', 'getall', 'namespaces', 're', 're_first', 
                    # 'register_namespace', 'remove', 'remove_namespaces', 'root', 'selectorlist_cls', 'text', 'type', 'xpath'
                    
                    #experience_lst = sel.xpath("//*[starts-with(@id,\"ember\")]/div[3]/ul/li/div/div[2]")

                    fields = ["about","experience","education","licenses_and_certifications",\
                             "volunteering_experience","skills","publications","honors_and_awards","languages",\
                             "organizations", "interests"]
                    info_lst = sel.xpath("//*[starts-with(@id,\"ember\")]/div")
                    
                    #//*[@id="ember56"]/div[3]/ul/li[5]/div/div[2]
                    fields_pos = []
                    for idx, info in enumerate(info_lst):
                        
                        if "id" in info.attrib:
                            if info.attrib["id"] in fields:
                                fields_pos.append((info.attrib["id"], idx))
                                
                     
                    for field, pos in fields_pos:
                        
                        if field != "about":
                            sel_field = info_lst[pos + 2]
                            items_lst = sel_field.xpath("ul/li")
                            if len(items_lst) > 0:
                                txt_field = [get_text(i.root) for i in items_lst]
                                data_person[field] = txt_field
                            else:
                                data_person[field] = [get_text(info_lst[pos + 2].root)]
                            
                        else:
                            data_person[field] = [get_text(info_lst[pos + 2].root)]
                            
                         
                    output_file = os.path.join(output_dir, "%s.json" % (f[:-5]))
                    with open(output_file, "w") as fd_out:
                        json.dump(data_person, fd_out, indent=4)

--> data_profiles_html/barbarastneves.html
Bárbara Neves
--> data_profiles_html/carolina_gadelha_858718175.html
Carolina Gadelha
--> data_profiles_html/danielabarreiroclaro.html
Daniela Claro
--> data_profiles_html/nataliacf1208.html
Natália Correia Freitas
--> data_profiles_html/da_rassi_b723aa61.html
Amanda Rassi
--> data_profiles_html/alineevers.html
Aline Evers
--> data_profiles_html/ticiana_linhares_42225016.html
Ticiana Linhares
--> data_profiles_html/livy_real.html
Livy R.
--> data_profiles_html/soareskarina.html
Karina Soares dos Santos
--> data_profiles_html/viviane_moreira_ufrgs.html
Viviane Moreira
--> data_profiles_html/clarissa_lenina_scandarolli.html
Clarissa Lenina Scandarolli
--> data_profiles_html/rossana.html
Rossana Cunha
--> data_profiles_html/aline_macohin.html
Aline Macohin
--> data_profiles_html/maria_josé_bocorny_finatto_694b74178.html
Maria José Bocorny Finatto
--> data_profiles_html/elisa_terumi_rubel_schneider.html
Elisa Terumi Rubel Schneider
--> data_profil